## Movie Recommendation System
&nbsp;

* At some point each one of us must have wondered where all the recommendations that Netflix, Amazon, Google give us, come from. We often rate products on the internet and all the preferences we express and data we share (explicitly or not), are used by recommender systems to generate, in fact, recommendations. The two main types of recommender systems are either collaborative or content-based filters: these two names are pretty self-explanatory, but let’s look at a couple of examples to better understand the differences between them. I will use movies as an example (because if I could, I would be watching movies/tv shows all the time), but keep in mind that this type of process can be applied for any kind of product you watch, listen to, buy, and so on.


<img src = 'images/movie.png'>

### Why Recommendation System ?

&nbsp;
&nbsp;

* Here's the list of advantages:
    * Increase in sales due to better personalized offers.
    * Enhanced customer experience.
    * More time spent on the plateform.
    * Customer retention.

&nbsp;
&nbsp;

* A recent study by Epsilon found that 90% of consumers find personalization appealing. Plus, a further 80% claim they are more likely to do business with a company when offered personalized experiences.

&nbsp;

* The study also found that these consumers are 10x more likely to become VIP customers, who make more than 15 purchases per year.

&nbsp;

* The moral of the story? If you’re interested in cross selling or serving personalized offers, a recommendation system is right for you.

### Recommendation at Netflix.com

&nbsp;

<img src='images/nf2.jpg'>

### Types of Recommendations System

&nbsp;
&nbsp;

* Content Based Filtering

&nbsp;

* Collaborative Filtering


### Content Based Filtering

&nbsp;

<img src = 'images/cn.png' style="width:500px">

&nbsp;


* Content based filtering uses characteristics or properties of an item to serve recommendations. Characteristic information includes:

    * Characteristics of Items (Keywords and Attributes)
    * Characteristics of Users (Profile Information)
    
&nbsp;

* Let’s use a movie recommendation system as an example. Characteristics for the item Harry Potter and the Sorcerer’s Stone might include:

    * Director Name – Chris Columbus
    * Genres – Adventure, Fantasy, Family (IMDB)
    * Stars – Daniel Radcliffe, Rupert Grint, Emma Watson
    
&nbsp;

<img src = 'images/content-based-filtering-harry-potter.png' style="width:800px" >

&nbsp;

* A content based recommender system can now serve the user:
    * More Harry Potter Movies
    * More Adventure, Family, or Fantasy Movies
    * More Chris Columbus Movies
    * More Daniel Radcliffe Movies
    
&nbsp;

* Of course, the list is not exhaustive. Once the user makes choices, the recommender system can serve more targeted results.

&nbsp;

* Let’s say they choose Swiss Army Man next. It’s safe to assume the user likes movies starring Daniel Radcliffe. The system tracks these choices and begins to recommend Daniel Radcliffe films.

&nbsp;

* The system may also show the user more Harry Potter movies. The hypothesis is that if a user liked an item in the past, they might like similar items in the future.

&nbsp;

* Content based filtering systems can also serve users items based on users’ profiles. You can create user profiles based on historical actions. You can also ask users upfront about their interests and preferences.

&nbsp;

* Pro Tip: Using a hybrid recommender system allows you to combine elements of both systems. In general, that means elements of one system can remedy the pitfalls of the other.


* In this project I am going to implement a basic content based recommendation system. The dataset used in this project is [Movielens](https://grouplens.org/datasets/movielens/100k/)  100k dataset.

&nbsp;

#### About MovieLens Dataset

&nbsp;

* One of the most common datasets that is available on the internet for building a Recommender System is the MovieLens DataSet. This version of the dataset that I'm working with (100k) contains 100,000 anonymous ratings of approximately 1700 movies made by 1000 MovieLens users who joined MovieLens in 2000.

&nbsp;

* The data was collected by GroupLens researchers over various periods of time, depending on the size of the set. This 100k version was released on February 2003. Users were selected at random for inclusion. All users selected had rated at least 20 movies. Each user is represented by an id, and no other information is provided.

In [4]:
# importing libraries 
import pandas as pd
import numpy as np

In [ ]:
# importing user data from the zip file
user_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names = user_cols, encoding = 'latin-1')

# importing movie ratings from the zip file
ratings_cols = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names = ratings_cols, encoding = 'latin-1')

# importing movies data from the zip file
movies_cols = ['movie_id','title','release_date','video_release_date','imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names = movies_cols,usecols = range(5),
                     encoding = 'latin-1')

In [ ]:
# importing genre dataset 
genres_list = ['unknown','Action','Adventure','Animation','Children','Comedy','Crime',
               'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
               'Romance','Sci-Fi','Thriller','War','Western']
genre = pd.read_csv('ml-100k/u.item', sep='|',names = genres_list,usecols = range(5,24),encoding = 'latin-1')

In [ ]:
# dropping redundant columns
movies.drop(['video_release_date','imdb_url'],inplace=True,axis = 1)
ratings.drop('unix_timestamp',axis = 1,inplace=True)

In [ ]:
# merge all the dataset into one whole dataset
dataset = pd.merge(pd.merge(movies, ratings),users)

In [ ]:
# top 20 most rated movies
dataset[['title','rating']].sort_values('rating', ascending=False).head(20)

### Cosine Similarity

&nbsp;

* We all are familiar with vectors: they can be 2D, 3D or whatever-D. Let’s think in 2D for a moment, because it’s easier to picture in our mind, and let’s refresh the concept of dot product first. The dot product between two vectors is equal to the projection of one of them on the other. Therefore, the dot product between two identical vectors (i.e. with identical components) is equal to their squared module, while if the two are perpendicular (i.e. they do not share any directions), the dot product is zero. Generally, for n-dimensional vectors, the dot product can be calculated as shown below.

&nbsp;

<img src ='images/cos.png' >

&nbsp;

* The dot product is important when defining the similarity, as it is directly connected to it. The definition of similarity between two vectors u and v is, in fact, the ratio between their dot product and the product of their magnitudes.

&nbsp;

<img src ='images/cos2.png' >

&nbsp;

* By applying the definition of similarity, this will be in fact equal to 1 if the two vectors are identical, and it will be 0 if the two are orthogonal. In other words, the similarity is a number bounded between 0 and 1 that tells us how much the two vectors are similar. Pretty straightforward!


In [ ]:
# Totol movies in terms of genre
genre.sum().sort_values(ascending=False)

In [ ]:
# Applying Cosine Similarity on genre dataset since it is already in sparse matrix form
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(genre)

In [ ]:
# Let's generate recommendation based on those similarity score of movies

# first create a list of titles
titles = movies['title']

# Create a series as key and value with key being movies title and value being it's indices
indices = pd.Series(movies.index,index=movies['title'])

# creating the recommendation function
def movie_recommendation(title):
    #gets the index of the recieved title
    index = indices[title]
    #gets the similarity scores for the movies similar to the one at the index 
    sim_scores = list(enumerate(cosine_sim[index]))
    # sorts the score in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # stores top 20 score
    sim_scores = sim_scores[1:21]
    # store the indices of the top 20 score
    movie_indices = [i[0] for i in sim_scores]
    # return the list of movies on that indices
    return titles.iloc[movie_indices]

In [ ]:
movie_recommendations('Toy Story (1995)').head(20)